In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
from datetime import timedelta
import zipfile
import folium
import json
import joblib
import pyodbc

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [3]:
raw_requests = pd.read_csv('data/open_requests.csv')

In [4]:
def fix_date(datestr , delim):
    if pd.isna(datestr):
        return np.nan
    year, month, day = datestr.split(delim)[0].split('-')
    return datetime(int(year), int(month), int(day))

In [5]:
with zipfile.ZipFile("data/current_payments.zip",mode="r") as curr_pay_ref:
    curr_pay_ref.extractall()

In [6]:
raw_payments = pd.read_csv('current_payments.csv')
raw_payments.date_trans_start = raw_payments.date_trans_start.apply(fix_date, args=(' ',))

In [7]:
meter_raw = pd.read_csv('data/parking_meters.csv')

sunday_meters = meter_raw[meter_raw.config_name == 'Sunday Mode'].index
meter_raw.drop(sunday_meters, inplace=True)

In [8]:
meter_df = gpd.GeoDataFrame(meter_raw, 
                            geometry=gpd.points_from_xy(meter_raw.lng, meter_raw.lat), 
                            crs='EPSG:4326').to_crs(epsg=3857)
requests_df = gpd.GeoDataFrame(raw_requests, 
                               geometry=gpd.points_from_xy(raw_requests.lng, raw_requests.lat), 
                               crs='EPSG:4326').to_crs(epsg=3857)

In [9]:
ZIPS = 'spatial/zips.geojson'
zip_gdf = gpd.read_file(ZIPS).to_crs(epsg=3857)

In [10]:
valid_zips = meter_df.sjoin(zip_gdf, lsuffix='_meter', rsuffix='_zip').ZIP.unique()

requests_local_df = zip_gdf.set_index('ZIP').loc[valid_zips,:].sjoin(requests_df).reset_index().drop('index_right',
                                                                                                     axis=1)
requests_local_df.geometry = gpd.points_from_xy(requests_local_df.lng, requests_local_df.lat)
requests_local_df.crs = 'EPSG:4326'

In [11]:
VALID_COLUMNS = ['service_request_id','pole', 'geometry', 'ZIP', 'date_requested', 'service_name', 'lat_left', 'lng_left',
                 'comm_plan_name', 'case_age_days']

meter_df.geometry = meter_df.buffer(50)
valid_requests = meter_df.to_crs('EPSG:4326').sjoin(requests_local_df)
valid_requests.date_requested = valid_requests.date_requested.apply(fix_date, args=('T',))
valid_requests = valid_requests.loc[:, VALID_COLUMNS].reset_index(drop=True)

valid_requests

service_request_id      pole  \
0                  124835    1-1004   
1                  124835    1-1006   
2                  124835   FR-1019   
3                 3896656    1-1004   
4                 3896656    WD-105   
...                   ...       ...   
74185             3971875  WSL-200S   
74186             2684880  WSL-200S   
74187             3817657  WSL-200S   
74188             3908772   WUV-111   
74189             2428392   WUV-111   

                                                geometry    ZIP  \
0      POLYGON ((-117.16348 32.71590, -117.16348 32.7...  92101   
1      POLYGON ((-117.16348 32.71604, -117.16348 32.7...  92101   
2      POLYGON ((-117.16419 32.71604, -117.16419 32.7...  92101   
3      POLYGON ((-117.16348 32.71590, -117.16348 32.7...  92101   
4      POLYGON ((-117.16321 32.71564, -117.16321 32.7...  92101   
...                                                  ...    ...   
74185  POLYGON ((-117.16474 32.71024, -117.16475 32.7...  92101   
74186  POLYGON ((-117.16474 32.71024, -117.16475 32.7...  92101   
74187  POLYGON ((-117.16474 32.71024, -117.16475 32.7...  92101   
74188  POLYGON ((-117.16329 32.74817, -117.16330 32.7...  92103   
74189  POLYGON ((-117.16329 32.74817, -117.16330 32.7...  92103   

      date_requested                             service_name   lat_left  \
0         2016-12-14                    Sidewalk Repair Issue  32.715904   
1         2016-12-14                    Sidewalk Repair Issue  32.716037   
2         2016-12-14                    Sidewalk Repair Issue  32.716039   
3         2022-08-27                   Shared Mobility Device  32.715904   
4         2022-08-27                   Shared Mobility Device  32.715643   
...              ...                                      ...        ...   
74185     2022-10-28                 Street Light Maintenance  32.710241   
74186     2019-08-06  Development Services - Code Enforcement  32.710241   
74187     2022-06-24                 Street Light Maintenance  32.710241   
74188     2022-09-06                                  Parking  32.748166   
74189     2018-10-22                    Sidewalk Repair Issue  32.748166   

         lng_left comm_plan_name  case_age_days  
0     -117.163929       Downtown           2219  
1     -117.163930       Downtown           2219  
2     -117.164639       Downtown           2219  
3     -117.163929       Downtown            137  
4     -117.163656       Downtown            137  
...           ...            ...            ...  
74185 -117.165192       Downtown             75  
74186 -117.165192       Downtown           1254  
74187 -117.165192       Downtown            201  
74188 -117.163743         Uptown            127  
74189 -117.163743         Uptown           1542  

[74190 rows x 10 columns]

In [12]:
#csv file used to in SQL queries
#dropping geometry saves significant storage costs
valid_requests.drop('geometry',axis=1).to_csv('data/new_requests.csv', encoding='utf-8', index=False)

In [13]:
def categorize_differences(diff, low_end, high_end):
    if diff > high_end:
        return "High Priority"
    else:
        return "Low Priority"
    
def get_sql_table(curs,tbl_name,skma='dbo'):
    #get row values
    row_arr = []
    rows = curs.execute('SELECT * FROM {}.{}'.format(skma, tbl_name)).fetchall()
    for row in rows:
        row_arr.append(np.array(row)) #change row from tuple --> array
    #get col names
    col_arr = []
    cols = curs.columns(table=tbl_name,schema=skma)
    for col in cols:
        col_arr.append(col[3]) #position 3 contains col name
    
    return pd.DataFrame(data=row_arr, columns=col_arr)

In [14]:
conn = pyodbc.connect('Trusted_Connection=yes', driver = '{SQL Server}',
                      server = 'LAPTOP-FI121EPT\SQLEXPRESS', database = 'parking')
cursor = conn.cursor()

In [15]:
valid_df = get_sql_table(cursor,'new_aggregated_data', 'dbo')

In [29]:
rf_mdl = joblib.load('forest.pkl')

In [30]:
service_categories = [['COVID-19', 'Dead Animal',
       'Development Services - Code Enforcement', 'Encampment',
       'Environmental Services Code Compliance', 'Graffiti',
       'Graffiti - Code Enforcement', 'Homeless Outreach',
       'Illegal Dumping', 'Missed Collection', 'Other',
       'Oversized Vehicle', 'Parking', 'Pavement Maintenance', 'Pothole',
       'ROW Maintenance', 'Right-of-Way Code Enforcement',
       'Shared Mobility Device', 'Sidewalk Repair Issue', 'Stormwater',
       'Stormwater Code Enforcement', 'Street Flooded',
       'Street Light Maintenance', 'Street Sweeping',
       'Traffic Engineering', 'Traffic Sign Maintenance',
       'Traffic Signal Issue', 'Traffic Signal Timing',
       'Trash/Recycling Collection', 'Tree Maintenance',
       'Waste on Private Property', 'Weed Cleanup'],[1, 2, 3, 4, 5, 6, 7,
       8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 30, 31]]

In [31]:
col_names = np.concatenate((service_categories[0], service_categories[1], ['payment_before']))

X = valid_df.loc[:,['service_name', 'payment_before', 'pole']]
one_hotter = OneHotEncoder(categories=service_categories,
                           handle_unknown='ignore',
                           sparse=False)
normer = StandardScaler()
sig_transformer = ColumnTransformer([('categories', one_hotter, ['service_name', 'pole']),
                                     ('numerical', normer, ['payment_before'])])
X = pd.DataFrame(sig_transformer.fit_transform(X), columns=col_names)

In [33]:
current_preds = rf_mdl.predict(X)
valid_df['predicted prio'] = current_preds

D:\anaconda\envs\parking\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [36]:
valid_df.to_csv('predictions/new_request_preds.csv', encoding='utf-8', index=False)
conn.close()